# FIBO MCP Ablation Study

Compare Claude Opus 4.5 and GPT-5.2 responses with and without FIBO MCP.

In [1]:
QUESTIONS = [
    "What is money?",
    "What is a country?",
    "What is corporate?",
]

In [2]:
import os
import json
import asyncio
from pathlib import Path

import pandas as pd

from openai import AsyncOpenAI
from claude_agent_sdk import ClaudeAgentOptions, query
from claude_agent_sdk.types import AssistantMessage, ResultMessage, TextBlock, ToolUseBlock

from dotenv import load_dotenv
load_dotenv()

# Remote MCP endpoint
FIBO_MCP_URL = os.environ["FIBO_MCP_URL"]
FIBO_MCP_KEY = os.environ["FIBO_MCP_KEY"]

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
# use `claude setup-token` to get a new OAuth token
CLAUDE_CODE_OAUTH_TOKEN = os.environ["CLAUDE_CODE_OAUTH_TOKEN"]

In [3]:
# claude-agent-sdk is equivalent to claude code

async def query_claude_code(question: str, use_fibo: bool=False) -> str:
    """Query Claude with FIBO MCP tool access."""
    mcp_servers = None
    if use_fibo:
        mcp_servers = {
            "fibo-mcp": {
                "type": "http",
                "url": FIBO_MCP_URL,
                "headers": {"Authorization": f"Bearer {FIBO_MCP_KEY}"},
            },
        }
    options = ClaudeAgentOptions(
        model="claude-opus-4-5-20251101",
        system_prompt="Use fibo MCP tool to answer questions about financial concepts.",
        mcp_servers=mcp_servers,
        allowed_tools=["mcp__fibo-mcp__*"],
        permission_mode="bypassPermissions",
    )
    async for message in query(prompt=question, options=options):
        if isinstance(message, ResultMessage):
            return message.result or ""
        
    return ""

In [4]:
openai_client = AsyncOpenAI(api_key=OPENAI_API_KEY)

async def query_gpt52(question: str, use_fibo: bool=False) -> str:
    """Query GPT-5.2 with FIBO MCP tool access."""
    tools = None
    if use_fibo:
        tools = [
            {
                "type": "mcp",
                "server_label": "fibo",
                "server_url": FIBO_MCP_URL,
                "headers": {"Authorization": f"Bearer {FIBO_MCP_KEY}"},
                "require_approval": "never",
            },
        ]
    response = await openai_client.responses.create(
        model="gpt-5.2",
        instructions="Use fibo MCP tool to answer questions about financial concepts.",
        tools=tools,
        input=question,
    )
    return response.output_text

In [5]:
def write_jsonl(results:list, llm_provider:str):
    with open(f"{llm_provider}_results.jsonl", "w") as f:
        for i, question in enumerate(QUESTIONS):
            # Without FIBO
            f.write(json.dumps({
                "question": question,
                "with_fibo": False,
                "answer": results[i*2]
            }) + "\n")

            # With FIBO
            f.write(json.dumps({
                "question": question,
                "with_fibo": True,
                "answer": results[i*2+1]
            }) + "\n")

In [6]:
from IPython.display import display, Markdown

def display_markdown(results: list):
    md = []

    for i, question in enumerate(QUESTIONS):
        # Question header
        md.append(f"# {question}\n")

        # Side-by-side comparison using HTML table
        md.append('<table>')
        md.append('<tr>')
        md.append('<th width="50%">Raw</th>')
        md.append('<th width="50%">With FIBO MCP</th>')
        md.append('</tr>')
        md.append('<tr>')
        md.append(f'<td valign="top">\n\n{results[i*2]}\n\n</td>')
        md.append(f'<td valign="top">\n\n{results[i*2+1]}\n\n</td>')
        md.append('</tr>')
        md.append('</table>\n')

    display(Markdown('\n'.join(md)))

In [7]:
tasks = []
for question in QUESTIONS:
    tasks.append(query_gpt52(question))
    tasks.append(query_gpt52(question, use_fibo=True))

openai_results = await asyncio.gather(*tasks)

In [8]:
display_markdown(openai_results)

# What is money?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

Money is anything that is **widely accepted as payment** for goods and services and for settling debts. In finance and economics, it’s typically defined by what it *does*:

- **Medium of exchange:** lets people trade without barter (you don’t need a direct swap).
- **Unit of account:** provides a common way to quote prices and keep financial records (e.g., dollars, euros).
- **Store of value:** can be saved and used later (though inflation can erode this).
- **Standard of deferred payment:** can be used to repay debts over time.

**Forms of money** include:
- **Commodity money:** has intrinsic value (e.g., gold).
- **Fiat money:** value comes from government decree and public trust (most modern currencies).
- **Bank/credit money:** deposits created through the banking system (checking account balances).
- **Digital money:** electronic representations of fiat; **cryptocurrencies** may function as money if broadly accepted, but often behave more like speculative assets today.

If you tell me the context (personal finance, macroeconomics, banking, crypto), I can tailor the definition and examples.

</td>
<td valign="top">

In FIBO (the Financial Industry Business Ontology), **money** is represented primarily as **currency**:

- **Currency (FIBO class):** `fibo:FND/Accounting/CurrencyAmount/Currency`
- **Definition (from FIBO / ISO 4217):** a **“medium of exchange value, defined by reference to the geographical location of the monetary authorities responsible for it.”**
- **Synonyms in FIBO:** *currency unit*, *monetary unit*

In plain terms: **money is an accepted medium of exchange and unit of value—typically issued/defined by a monetary authority (e.g., a central bank) and tied to a jurisdiction (like USD for the United States, JPY for Japan).**

</td>
</tr>
</table>

# What is a country?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

A **country** is a politically organized territory that is recognized as a distinct entity and generally has:

- **Defined territory** (land and sometimes surrounding waters)
- **A permanent population**
- **A government** that makes and enforces laws
- **Sovereignty** (the ability to govern itself without being controlled by another state)
- **International recognition** (often, but not always, acknowledged by other countries and able to engage in foreign relations)

In everyday use, “country” usually means a **sovereign state** (like Japan or Kenya). Sometimes the term is also used for **constituent countries** or regions with a high degree of self-rule but not full sovereignty (like Scotland within the UK).

</td>
<td valign="top">

In FIBO (the Financial Industry Business Ontology), what people commonly call a **country** is modeled as a **sovereign state**:

- **FIBO class:** `fibo:BE/GovernmentEntities/GovernmentEntities/SovereignState` (label: **“sovereign state”**)

A **sovereign state (country)** is a political entity that is regarded as having:
- a defined territory,
- a permanent population,
- a governing authority, and
- sovereignty (i.e., it is not legally subordinate to another state and can act independently in international relations).

If you tell me whether you mean “country” in the *political* sense (sovereign state) or in the *geographic/region* sense (e.g., “country of risk,” “country code,” “country of incorporation”), I can map it to the most precise FIBO concept.

</td>
</tr>
</table>

# What is corporate?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

“Corporate” generally refers to anything related to a **corporation**—a legally recognized business entity that is separate from its owners.

In finance and business, “corporate” is used in a few common ways:

- **Corporate (adjective):** pertaining to companies/corporations (e.g., *corporate finance*, *corporate governance*, *corporate bonds*).
- **A corporate (noun, informal):** a corporation or large company (e.g., “working for a corporate”).
- **Corporate finance:** the area of finance focused on how companies raise money (debt/equity), invest it (projects/capex), and return value to owners (dividends/buybacks).
- **Corporate bonds/credit:** debt issued by companies to investors.

If you mean it in a specific context (e.g., “corporate action,” “corporate account,” or “corporate bonds”), tell me the phrase and I’ll define it precisely.

</td>
<td valign="top">

In a business/finance context, **“corporate”** means **relating to a corporation**—i.e., an incorporated legal entity that is legally separate from its owners.

In the Financial Industry Business Ontology (FIBO), the closest core concept is:

- **fibo:BE/LegalEntities/CorporateBodies/Corporation** (label: **“corporation”**)

Common uses of “corporate”:
- **Corporate entity / corporate body**: the company as a legal person (it can own assets, sign contracts, sue/be sued).
- **Corporate governance**: how the corporation is directed/controlled (board, officers, bylaws).
- **Corporate finance**: funding/financial decisions of corporations.
- **Corporate bonds / corporate debt**: debt securities issued by corporations.

</td>
</tr>
</table>


In [9]:
tasks = []
for question in QUESTIONS:
    tasks.append(query_claude_code(question))
    tasks.append(query_claude_code(question, use_fibo=True))

claude_code_results = await asyncio.gather(*tasks)

In [10]:
display_markdown(claude_code_results)

# What is money?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

Money is a medium of exchange that is widely accepted for the payment of goods, services, and debts. It serves several key functions in an economy:

## Core Functions of Money

1. **Medium of Exchange** - Money eliminates the need for barter (trading goods directly for other goods). Instead of finding someone who has what you want AND wants what you have, you can simply use money to buy what you need.

2. **Unit of Account** - Money provides a standard measure of value, allowing us to compare the worth of different goods and services. Prices are expressed in monetary units (dollars, euros, yen, etc.).

3. **Store of Value** - Money allows people to save purchasing power for the future. You can earn money today and spend it later.

4. **Standard of Deferred Payment** - Money enables credit and debt by providing a way to specify future payments (loans, contracts, etc.).

## Characteristics of Good Money

- **Durability** - It should last over time
- **Portability** - Easy to carry and transfer
- **Divisibility** - Can be broken into smaller units
- **Uniformity** - Each unit is identical to others
- **Acceptability** - Widely recognized and trusted
- **Limited Supply** - Scarcity helps maintain value

## Types of Money

- **Commodity Money** - Has intrinsic value (gold, silver, salt historically)
- **Fiat Money** - Government-issued currency not backed by a physical commodity (most modern currencies)
- **Digital/Electronic Money** - Exists only in digital form (bank deposits, cryptocurrencies)

Money is essentially a social agreement—it works because people collectively trust and accept it as valuable for transactions.

</td>
<td valign="top">

Based on my research in FIBO (Financial Industry Business Ontology), here's what **money** is according to financial industry standards:

## Money in FIBO

FIBO defines the concept of money through two closely related terms:

### 1. **Currency**
> *"A medium of exchange value, defined by reference to the geographical location of the monetary authorities responsible for it."*

- **Source**: ISO 4217 (Codes for the representation of currencies and funds)
- **Synonyms**: "currency unit", "monetary unit"
- **Classification**: It is a type of **Measurement Unit** in FIBO's ontology

### 2. **Amount of Money** (also called "Cash")
> *"Amount of readily available cash in banknotes and coins"*

This represents an actual sum of money - the physical or readily available form of currency.

---

## Key Characteristics of Money in FIBO

| Aspect | Description |
|--------|-------------|
| **Function** | Medium of exchange value |
| **Authority** | Defined by geographical monetary authorities (e.g., central banks) |
| **Physical Form** | Banknotes and coins (when referring to cash) |
| **Measurement** | Expressed as a scalar quantity with a currency unit |

---

## Related Financial Concepts

FIBO also includes many money-related concepts such as:
- **Money Market Instruments** - short-term debt securities
- **Money Services Business** - entities that deal with currency exchange, money transmission, check cashing, etc.
- **Money Market Futures** - futures contracts with money market instruments as underlying assets

In summary, **money** in financial industry terms is fundamentally a **medium of exchange** that is issued and regulated by monetary authorities, and can be measured in specific currency units (like USD, EUR, GBP, etc.).

</td>
</tr>
</table>

# What is a country?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

A **country** is a distinct territorial body or political entity that is recognized as an independent nation. It typically has the following characteristics:

## Key Characteristics of a Country:

1. **Defined Territory** - A country has recognized geographical boundaries that separate it from other nations.

2. **Permanent Population** - People who live within those boundaries on a long-term basis.

3. **Government** - An organized political system that maintains order, provides services, and makes decisions for the population.

4. **Sovereignty** - The authority to govern itself without interference from other nations; the ability to make its own laws and conduct its own affairs.

5. **Recognition** - Typically acknowledged by other countries and international organizations (though this can be disputed in some cases).

## Additional Notes:

- Countries can vary greatly in size, from large nations like Russia or Canada to small ones like Vatican City or Monaco.
- They can have different types of governments: democracies, monarchies, republics, authoritarian regimes, etc.
- The terms "country," "nation," and "state" are often used interchangeably, though they can have slightly different meanings in political science (e.g., "nation" often refers to a group of people with shared culture/identity, while "state" emphasizes the political/governmental aspect).

Currently, there are approximately **195 recognized countries** in the world (193 UN member states plus 2 observer states: Vatican City and Palestine).

</td>
<td valign="top">

Based on the FIBO (Financial Industry Business Ontology), here's what a **country** is:

## Country (Sovereign State) in FIBO

In FIBO, the concept of "country" is formally represented as a **Sovereign State**:

### Definition
> A **non-physical juridical entity that is represented by one centralized government that has sovereignty over a geographic area**

### Key Characteristics
According to FIBO's explanatory note, a sovereign state is:
- A **legal entity** represented by one centralized government
- Has a **permanent population**
- Has a **defined territory**
- Has the **capacity to enter into relations with other sovereign states**

### Ontology Hierarchy
- **SovereignState** is a subclass of **Polity**
  - A **Polity** is defined as: *"a legal person that is a supranational entity, crown, state, or subordinate civil authority, such as a province, prefecture, county, municipality, city, or district representing the people of that entity"*
- **Polity** is a subclass of **GovernmentBody**
  - A **GovernmentBody** is: *"a formal organization that is an agency, instrumentality, or other body of a supranational, national, federal, state, or local government"*

### Location in FIBO
`fibo:BE/GovernmentEntities/GovernmentEntities/SovereignState`

This definition aligns with the international legal concept of statehood as established by the Montevideo Convention (1933), which defines the criteria for sovereign statehood in international law.

</td>
</tr>
</table>

# What is corporate?

<table>
<tr>
<th width="50%">Raw</th>
<th width="50%">With FIBO MCP</th>
</tr>
<tr>
<td valign="top">

## What is "Corporate"?

**Corporate** is a term that relates to corporations, large companies, or business organizations. Here are the key meanings:

### Primary Definitions:

1. **Relating to a corporation or business**: Describing something that belongs to, relates to, or is characteristic of a large company or business organization.
   - *Example*: "Corporate profits increased this quarter"

2. **Incorporated status**: A legal term meaning an entity that has been formed into an association and given rights and liabilities under the law, similar to an individual.

3. **Collective or shared**: Pertaining to a group as a whole rather than individuals within it.
   - *Synonyms*: collective, collaborative, united, shared

### Common Uses:

- **Corporate culture** – the values and behaviors within a company
- **Corporate headquarters** (often shortened to just "corporate") – the main office of a company
- **Corporate structure** – how a company is organized
- **Corporate responsibility** – a company's obligations to society

### Etymology:

The word comes from the Latin *"corporare"* meaning "to form into a body," derived from *"corpus"* meaning "body" – essentially a "body of people" united as one entity.

### In Finance:

Corporate also refers to **corporate bonds** – debt securities issued by companies to raise capital, as opposed to government bonds.

---

**Sources:**
- [CORPORATE Definition & Meaning - Merriam-Webster](https://www.merriam-webster.com/dictionary/corporate)
- [CORPORATE | Cambridge English Dictionary](https://dictionary.cambridge.org/us/dictionary/english/corporate)
- [Corporate - Definition, Meaning & Synonyms | Vocabulary.com](https://www.vocabulary.com/dictionary/corporate)
- [Corporation - Wikipedia](https://en.wikipedia.org/wiki/Corporation)

</td>
<td valign="top">

Based on my research in FIBO (Financial Industry Business Ontology), here's what **"Corporate"** means in the financial industry:

## Corporation (Body Corporate)

According to FIBO, a **Corporation** is defined as:

> *"A formal organization that is a legal entity (artificial person) distinct from its owners, created under the jurisdiction of the laws of a state or nation."*

### Key Characteristics

A corporation has **three distinguishing characteristics**:

1. **Separation of ownership from management and liability** - Its liability to creditors is limited to its resources (unlike partnerships and sole proprietorships)
2. **Contractual capacity** - The ability to negotiate contracts and own property
3. **Transferable ownership** - Ownership can be transferred irrespective of changes in membership or the lifetimes of its stockholders

### Governance
- A corporation is managed by or under the direction of a **board of directors**, which determines corporate policy
- **Officers** manage day-to-day affairs

---

## Types of Corporations in FIBO

| Type | Definition |
|------|------------|
| **For-Profit Corporation** | Corporation whose objective is to make money and ensure financial benefit exceeds expenses |
| **Not-for-Profit Corporation** | Corporation approved for educational, charitable, social, religious, civic or humanitarian purposes |
| **Stock Corporation** | Corporation with shareholders who receive ownership through shares of stock |
| **Publicly Held Company** | Corporation whose shares are traded and held publicly |
| **Privately Held Company** | Corporation whose shares are held by a family or small group and cannot be bought by the public |
| **Benefit Corporation** | Not-for-profit corporation set up to provide social benefit, without obligation to maximize shareholder return |

---

## Related Corporate Concepts

- **Corporate Bylaws** - Written rules for conduct of a corporation, adopted by the board of directors
- **Corporate Bond** - A bond issued by a company in order to raise financing
- **Corporate Actions** - Events or actions initiated by a corporation that affect its securities and stakeholders

</td>
</tr>
</table>
